In [6]:
# Table of liquid-phase density for ocean calculation [kg/m^3]
#     All taken from this website:
#     https://encyclopedia.airliquide.com/water#properties
const _lookup_liquid_rho::Dict{String, Float64} = Dict([
    ("H2O", 958.37 ),  # boiling
    ("CO2", 1178.4 ),  # triple
    ("H2" , 70.516 ),  # boiling
    ("CH4", 422.36 ),  # boiling
    ("CO" , 793.2  ),  # boiling
    ("N2" , 806.11 ),  # boiling
    ("NH3", 681.97 ),  # boiling
    ("SO2", 1461.1 ),  # boiling
])

Dict{String, Float64} with 8 entries:
  "N2"  => 806.11
  "CO"  => 793.2
  "CH4" => 422.36
  "NH3" => 681.97
  "H2O" => 958.37
  "SO2" => 1461.1
  "CO2" => 1178.4
  "H2"  => 70.516

In [27]:
function liquid_rho(name::String)::Float64
    if name in keys(_lookup_liquid_rho)
        return _lookup_liquid_rho[name]
    else
        return 1e99
    end
end

liquid_rho (generic function with 1 method)

In [ ]:
function ocean_layers(sigs::Dict{String, Float64},
                        fOB::Float64, hCS::Float64, Rpl::Float64)::Array

    # Areas [m2]
    aPL::Float64 = 4 * pi * Rpl^2    # planet area
    aOB::Float64 = aPL * fOB         # ocean basin

    # Total capacity of oceans [m3]
    v_cap_OB::Float64 = aOB * hCS

    # Get liquid densities
    liqs::Array{String, 1} = collect(keys(sigs))
    rhos::Array{Float64,1} = Float64[ liquid_rho(l) for l in liqs]

    # Sort liquids by decreasing density
    mask::Array{Int, 1} = reverse(sortperm(rhos))

    # Loop through liquids, filling oceans first
    v_tot_OB::Float64 = 0.0 # current amount of all things in oceans
    v_fill_OB::Array{Float64,1} = zero(rhos)  # volume of each puddle in oceans
    v_fill_CS::Array{Float64,1} = zero(rhos)  # volume of each puddle on continents
    output::Array = []
    for (j,i) in enumerate(mask)
        # total volume of rain from this liquid
        v_liq = sigs[liqs[i]] * aPL / rhos[i]

        # volume currently contained within oceans
        v_tot_OB = sum(v_fill_OB)

        # oceans full already?
        if v_tot_OB >= v_cap_OB
            # add all liquid to surface
            v_fill_CS[i] = v_liq

        # oceans not full yet
        else
            # oceans *will* be filled by this liquid
            if v_liq > v_cap_OB - v_tot_OB
                v_fill_OB[i] = v_cap_OB - v_tot_OB
                v_fill_CS[i] = v_liq - v_fill_OB[i]

            # oceans can contain all of this liquid, with some space left over
            else
                v_fill_OB[i] = v_liq
            end
        end

        # output array with consolidated info
        #   surface liquid is divided by total area, since it covers oceans+continents
        push!(output, (j, liqs[i], v_fill_OB[i]/aOB, v_fill_CS[i]/aPL))
    end

    return output
end # end ocean_layers

ocean_layers (generic function with 2 methods)

In [55]:
earth_annu = 977.4 # H2O rain per year, kg/m^2

977.4

In [56]:
sigs = Dict([
    ("H2O",earth_annu*3000),
    ("CO2",1.0),
    ("H2S",300.0),
    ])

f = 0.6    # 1
h = 3600.0 # m
r = 6371e3 # m

ocean_layers(sigs, f, h, r)

3-element Vector{Any}:
 (1, "H2S", 5.000000000000001e-97, 0.0)
 (2, "CO2", 0.001414347137361394, 0.0)
 (3, "H2O", 3599.9985856528624, 899.5707433253542)